<a href="https://colab.research.google.com/github/ken001111/Naver-Boostcamp-2024/blob/main/(%EA%B8%B0%EB%B3%B8_2)_Linear_Regression_(%EB%AC%B8%EC%A0%9C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Linear Regression 과제**
---

### **1) 과제 개요**

- pytorch를 이용하여 Boston House Prices데이터에 대해 주택 가격을 예측하는 선형 회귀 분석을 시행합니다.

### **2) 과제 진행 목적 및 배경**

- pytorch를 이용하여 선형 회귀 분석을 위한 모델을 구축하고, 이를 통해 선형 회귀 분석을 실시하는 방법을 공부합니다.

### **3) 과제 수행으로 얻어갈 수 있는 역량**

- 주어진 데이터를 pytorch 학습에 맞게 전처리할 수 있다.

- 선형 회귀 분석에 필요한 pytorch 구성 요소들을 구현할 수 있다.

- 실제 데이터에 대해 선형 회귀 분석을 실시할 수 있다.

### **4) 과제 핵심 내용**

- Boston House Prices 데이터를 선형회귀에 맞게 전처리합니다.

- 선형 회귀 모델을 pytorch로 구현하고, 이를 학습시킵니다.

- 머신러닝을 통해 얻은 회귀계수를 analytic solution과 비교합니다.

### **5) 데이터셋 개요 및 저작권 정보**

- 사용 데이터셋: [Boston House Prices](https://www.kaggle.com/datasets/arunjathari/bostonhousepricedata)
  - 보스턴 시의 범죄율, 재산세율, 본인 소유의 주택가격의 중앙값등의 측정된 지표를 포함한 데이터셋입니다.
- 저작권 정보: [CC0 1.0 Universal](https://creativecommons.org/publicdomain/zero/1.0/)

### **6) Required Package**

```python
scikit-learn >= 1.4.2
torch >= 2.3.0
pandas >= 2.2.2
numpy >= 1.26.4
```

---
### **과제 진행 방법**
`[CODE START]`와 `[CODE END]` 사이의 코드들을 완성하는 것을 목표로 합니다.

---
## **Boston House Prices 데이터를 다운받고 전처리하기**

#### 문제 1. 데이터셋 개요에 첨부된 [링크](https://www.kaggle.com/datasets/arunjathari/bostonhousepricedata)에서 데이터를 다운받고, 변수 `data`에 데이터를 저장하세요.
힌트: pandas 라이브러리의 read_csv 함수를 이용하세요.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import pandas as pd
# [CODE START]
data = pd.read_csv("Boston-house-price-data.csv", sep = ',', header = 0)
# [CODE END]

올바르게 불러왔다면, 아래 코드가 에러없이 잘 수행됩니다.

In [13]:
assert data.shape == (506, 14)
print("Data is loaded correctly!")
data.head()

Data is loaded correctly!


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


MEDV 변수를 제외한 나머지 예측 변수들을 `X`에 저장하고, 종속변수인 MEDV를 `y`에 저장합니다.

이 때, 선형회귀모델의 성능을 높이기 위해 예측 변수의 값들을 정규화하였습니다.

In [34]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(data.drop(['MEDV'], axis=1)), columns=data.columns[:-1])
y = data.MEDV

이제, 임의로 학습용 데이터와 평가용 데이터를 8:2로 나눕니다.

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, y_train = shuffle(X_train, y_train, random_state=42)

각자 환경에 맞는 device를 설정합니다.

In [36]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

#### 문제 2. 변수 `X_train_tensor`, `y_train_tensor`, `X_test_tensor`, `y_test_tensor`에 각각 학습을 위한 텐서와 평가를 위한 텐서를 저장하세요.
- 이 때, 각 텐서가 환경에 맞는 device (cpu or gpu)상에 저장되도록 작성하세요.
- 추후 편의를 위해 `y`에 대한 텐서의 차원은 (데이터 개수, 1)의 형태가 되도록 작성하세요.

In [41]:
# [CODE START]
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)
# [CODE END]

올바르게 저장했다면, 아래 코드가 에러없이 잘 수행됩니다.

In [42]:
assert X_train_tensor.shape == (404, 13)
assert y_train_tensor.shape == (404, 1)
assert X_test_tensor.shape == (102, 13)
assert y_test_tensor.shape == (102, 1)

assert X_train_tensor.device.type == device
assert y_train_tensor.device.type == device
assert X_test_tensor.device.type == device
assert y_test_tensor.device.type == device
print("Tensor is loaded correctly!")

Tensor is loaded correctly!


학습을 위한 데이터셋 객체와 데이터로더 객체를 생성합니다. 이 때, 배치 사이즈는 기본적으로 32로 설정하였습니다

In [43]:
from torch.utils.data import DataLoader, TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

---
## **선형회귀모델 작성하고 학습하기**

#### 문제 3. TODO 부분을 채워 다중선형회귀 모델을 위한 클래스를 완성하세요.
- `__init__` 함수 내에서 인스턴스 변수 `linear`에 `input_size`에 저장된 차원의 데이터를 받아 1차원의 데이터를 반환하는 선형 layer를 저장하세요.

- 해당 네트워크가 올바른 출력값을 내뱉도록 `forward` 함수를 완성하세요.

In [45]:
import torch.nn as nn
class MultipleLinearRegression(nn.Module):
    def __init__(self, input_size):
        super(MultipleLinearRegression, self).__init__()
        # [CODE START]
        self.linear = nn.Linear(input_size, 1)
        # [CODE END]

    def forward(self, x):
        # [CODE START]
        y = self.linear(x)
        return y
        # [CODE END]

올바르게 작성했다면, 아래 코드가 에러없이 잘 수행됩니다.

In [46]:
input_size = X_train_tensor.shape[1]
model = MultipleLinearRegression(input_size)
model.to(device)

assert model(X_test_tensor).shape == (102, 1)
print("Model is implemented correctly!")

Model is implemented correctly!


#### 문제 4. `criterion` 변수에 MSE를 손실함수로 하는 객체를 저장하세요.

In [48]:
# [CODE START]
criterion = nn.MSELoss()
# [CODE END]

#### 문제  5. `optimizer` 변수에 SGD로 최적화하는 객체를 저장하세요.
이 때, learning rate은 0.001로 설정하세요.

In [49]:
import torch.optim as optim
# [CODE START]
optimizer = optim.SGD(model.parameters(), lr=0.001)
# [CODE END]

이제, 지금까지 작성한 코드를 바탕으로 다중선형회귀 모형을 학습합니다.

In [50]:
import numpy as np

num_epochs = 2000 # epoch를 2000으로 설정
model.train() # 학습을 위해 모델이 gradient를 저장하도록 설정
for epoch in range(num_epochs):
    epoch_loss = 0.0
    data_num = 0
    for inputs, targets in train_loader: # 각 배치마다 반복
        optimizer.zero_grad() # 옵티마이저의 gradient 초기화
        outputs = model(inputs) # 데이터를 넣었을 때의 모델의 출력값 저장
        loss = criterion(outputs, targets) # MSE 손실 계산
        loss.backward() # gradient descent 수행
        optimizer.step() # SGD 방식의 최적화 진행
        epoch_loss += loss.item()*inputs.shape[0]
        data_num += inputs.shape[0]
    if (epoch+1) % 100 == 0: # 100번의 epoch마다 학습 데이터의 손실함수 출력
        print(f"Epoch {epoch+1}, Loss(RMSE): {np.sqrt(epoch_loss/data_num)}")

Epoch 100, Loss(RMSE): 5.095888669571717
Epoch 200, Loss(RMSE): 4.726424116327097
Epoch 300, Loss(RMSE): 4.693609521362971
Epoch 400, Loss(RMSE): 4.678734916035684
Epoch 500, Loss(RMSE): 4.669959398083812
Epoch 600, Loss(RMSE): 4.664915391793676
Epoch 700, Loss(RMSE): 4.661665652219798
Epoch 800, Loss(RMSE): 4.659716489218518
Epoch 900, Loss(RMSE): 4.658414507056033
Epoch 1000, Loss(RMSE): 4.656701920709057
Epoch 1100, Loss(RMSE): 4.656056704473851
Epoch 1200, Loss(RMSE): 4.6566309218264905
Epoch 1300, Loss(RMSE): 4.65500849767615
Epoch 1400, Loss(RMSE): 4.655125172040832
Epoch 1500, Loss(RMSE): 4.656030697778254
Epoch 1600, Loss(RMSE): 4.6547441564048215
Epoch 1700, Loss(RMSE): 4.654388374975508
Epoch 1800, Loss(RMSE): 4.654133822971111
Epoch 1900, Loss(RMSE): 4.65477729446455
Epoch 2000, Loss(RMSE): 4.6546491610841745


학습한 모델을 바탕으로 평가 데이터에 대한 RMSE를 아래 공식과 같이 계산합니다.

$$
\text{RMSE}(y, \hat{y}) = \sqrt{\frac{\sum_{i=0}^{N - 1} (y_i - \hat{y}_i)^2}{N}}
$$

In [51]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = criterion(y_pred, y_test_tensor)
    print(f"Test Loss(RMSE): {np.sqrt(test_loss.item())}")

Test Loss(RMSE): 4.936456690788868


#### 문제 6. 학습한 모델로부터 intercept에 대한 값을 변수 `b0`에, 나머지 변수에 대한 회귀계수 값을 변수 `b1`에 저장하세요.

힌트: `torch.nn.Module.parameters()` 함수는 PyTorch에서 신경망 모델의 모든 학습 가능한 파라미터들을 반환하는 함수입니다.

In [67]:
# [CODE START]
params = list(model.parameters())
b0 = params[0]
b1 = params[1]
# [CODE END]

In [68]:
print(f"intercept: {b0.cpu().data.numpy()}, other coef: {b1.cpu().data.numpy()}")

intercept: [[-0.9736214   0.68977416  0.2500494   0.7151077  -1.9947908   3.1248848
  -0.18068266 -3.040006    2.1975741  -1.7025492  -1.980231    1.124907
  -3.626526  ]], other coef: [22.480146]


---
## Analytic Solution과 비교하기

이제, 머신러닝을 통해 계산한 회귀계수와 공식을 통해 계산한 값을 비교하고자 합니다.

scikit-learn 라이브러리에서 `LinearRegression` 객체는 선형회귀모형을 적합하는 모형입니다.

In [60]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

아래와 같이 `LinearRegression` 객체를 생성하고 학습데이터를 바탕으로 적합하면 공식을 통해 회귀계수를 계산할 수 있습니다.

In [61]:
lm = LinearRegression()
lm.fit(X_train, y_train)

LinearRegression()

이렇게 적합한 모델을 가지고 학습 데이터와 평가 데이터의 RMSE를 계산할 수도 있습니다.

In [62]:
y_pred = lm.predict(X_train)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print(f"Train Loss(RMSE): {rmse}")

Train Loss(RMSE): 4.6520331848801675


In [63]:
y_pred = lm.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Test Loss(RMSE): {rmse}")

Test Loss(RMSE): 4.928602182665338


다음과 같이 `LinearRegression`객체의 인스턴스 변수 `intercept_`와 `coef_`에 접근함으로써 회귀계수를 추출할 수 있습니다.

In [64]:
print(f"intercept: {lm.intercept_}, tangent: {lm.coef_}")

intercept: 22.485268239316902, tangent: [-0.97149423  0.70155562  0.27675212  0.70653152 -1.99143043  3.11571836
 -0.17706021 -3.04577065  2.28278471 -1.79260468 -1.97995351  1.12649864
 -3.62814937]


두가지 방법으로 계산한 회귀계수의 차이를 계산한다면, 머신러닝을 통해 수치적인 방법으로 계산한 회귀계수의 정확성을 판단할 수 있습니다.

In [69]:
((lm.intercept_ - b0.cpu().detach().numpy())**2).sum()

6866.3823

In [70]:
((lm.coef_ - b1.cpu().detach().numpy())**2).mean()

527.9342940559543

## 콘텐츠 라이선스

<hr style="height:5px;border:none;color:#5F71F7;background-color:#5F71F7">

<font color='red'><b>WARNING</font> : 본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다. 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다. </b>